In [4]:
import numpy as np
from itertools import product
import matplotlib.pyplot as plt
from re import sub
from wolframclient.evaluation import WolframLanguageSession
from wolframclient.language import wl, wlexpr
from kerrgeopy.constants import *
import kerrgeopy
from numpy.polynomial import Polynomial

session = WolframLanguageSession()
session.evaluate(wlexpr("PacletDirectoryLoad[\"~/Documents/Wolfram Mathematica/BHPToolkit/KerrGeodesics\"]"))
session.evaluate(wl.Needs('KerrGeodesics`'))

## Separatrix

In [ ]:
num_values = int(1000)
sep_values = np.zeros((num_values,3))
i = 0
while i < num_values:
    parameters = np.round([np.random.rand(), np.random.rand(), np.random.rand()*2-1], \
                          decimals = 1)
    if parameters[0] != 1:
        sep_values[i] = parameters
        i += 1
        
def sub_zeros(txt):
    txt1 = sub("0\.0*,","0,",txt)
    txt1 = sub("0\.0*]","0]",txt1)
    txt1 = sub("1\.0*,","1,",txt1)
    txt1 = sub("1\.0*]","1]",txt1)
    return txt1

mathematica_sep_output = np.apply_along_axis(lambda x: \
                                        session.evaluate(wlexpr(
                                        sub_zeros("KerrGeoSeparatrix[{:},{:},{:}]".format(*x)))) \
                                        ,1,sep_values)

np.savetxt("sep_values.txt",sep_values,fmt="%.3f, %.3f, %.3f")
np.savetxt("mathematica_sep_output.txt",mathematica_sep_output)

## Constants

In [2]:
a_values = [0,0.5]
p_values = [12]
e_values = [0, 0.5, 1]
x_values = [-1,-0.5,0,0.5,1]

const_values = np.array(list(product(a_values,p_values,e_values,x_values)))

num_random_values = int(100)
random_values = np.zeros((num_random_values,4))
i = 0
while i < num_random_values:
    parameters = np.round([np.random.rand(), np.random.rand()*20, np.random.rand(), np.random.rand()*2-1], \
                          decimals = 3)
    if parameters[0] != 1 and parameters[1] > separatrix(parameters[0],parameters[2],parameters[3]):
        random_values[i] = parameters
        i += 1
        
const_values = np.concatenate([const_values,random_values])

mathematica_const_output = np.apply_along_axis(lambda x: \
                                         session.evaluate("Values[KerrGeoConstantsOfMotion[{:},{:},{:},{:}]]".format(*x)) \
                                         ,1,const_values)

np.savetxt("const_parameters.txt",const_values,fmt="%.3f, %.3f, %.3f, %.3f")
np.savetxt("mathematica_const_output.txt",mathematica_const_output)

## Frequencies

In [39]:
a_values = [0,0.5]
p_values = [12]
e_values = [0, 0.5]
x_values = [-1,-0.5,0.5,1]

freq_values = np.array(list(product(a_values,p_values,e_values,x_values)))

num_random_values = int(100)
random_values = np.zeros((num_random_values,4))
i = 0
while i < num_random_values:
    parameters = np.round([np.random.rand(), np.random.rand()*20, np.random.rand(), np.random.rand()*2-1], \
                          decimals = 3)
    if parameters[0] != 1 and parameters[1] > separatrix(parameters[0],parameters[2],parameters[3]):
        random_values[i] = parameters
        i += 1
        
freq_values = np.concatenate([freq_values,random_values])

mathematica_freq_output = np.apply_along_axis(lambda x: \
                                        session.evaluate(
            wlexpr("Values[KerrGeoFrequencies[{:},{:},{:},{:},\"Time\" -> \"Mino\"]]".format(*x))) \
                                        ,1,freq_values)


np.savetxt("freq_parameters.txt",freq_values,fmt="%.3f, %.3f, %.3f, %.3f")
np.savetxt("mathematica_freq_output.txt",mathematica_freq_output)

## Stable Solutions

In [2]:
a_values = [0,0.5]
p_values = [12]
e_values = [0, 0.5]
x_values = [-1,-0.5,0.5,1]

orbit_values = np.array(list(product(a_values,p_values,e_values,x_values)))

num_random_values = int(50)
random_values = np.zeros((num_random_values,4))
i = 0
while i < num_random_values:
    parameters = np.round([np.random.rand(), np.random.rand()*20, np.random.rand(), np.random.rand()*2-1], \
                          decimals = 3)
    if parameters[0] != 1 and parameters[2] != 1 and parameters[3] != 0 and parameters[1] > separatrix(parameters[0],parameters[2],parameters[3]):
        random_values[i] = parameters
        i += 1
        
orbit_values = np.concatenate([orbit_values,random_values])
np.savetxt("stable_orbit_parameters.txt",orbit_values,fmt="%.3f, %.3f, %.3f, %.3f")
times = np.array(range(10))
np.savetxt("stable_orbit_times.txt",times)

In [ ]:
for i, orbit in enumerate(orbit_values):
    session.evaluate(wlexpr("orbit = KerrGeoOrbitPhases[{},{},{},{}]".format(*orbit)))
    session.evaluate(wlexpr("{tr, ttheta, phir, phitheta} = Values[orbit[\"TrajectoryDeltas\"]];"))
    trajectory_deltas = np.zeros((10,4))
    for j,t in enumerate(times):
        trajectory_deltas[j] = np.array(session.evaluate(wlexpr(f'N[{{tr[{t}],ttheta[{t}],phir[{t}],phitheta[{t}]}}]')))
    np.savetxt("stable_solutions/trajectory"+str(i)+".txt",trajectory_deltas,delimiter=",")

## Stable Orbit

In [ ]:
for i, orbit in enumerate(orbit_values):
    session.evaluate(wlexpr("orbit = KerrGeoOrbit[{},{},{},{}]".format(*orbit)))
    session.evaluate(wlexpr("{t, r, theta, phi} = orbit[\"Trajectory\"]"))
    trajectory = np.zeros((10,4))
    for j,t in enumerate(times):
        trajectory[j] = np.array(session.evaluate(wlexpr(f'N[{{t[{t}],r[{t}],theta[{t}],phi[{t}]}}]')))
    np.savetxt("stable_orbits/trajectory"+str(i)+".txt",trajectory,delimiter=",")

## Plunging Solutions and Integrals (Complex Roots)

In [5]:
num_random_values = int(100)
orbit_values = np.zeros((num_random_values,4))
i = 0

while i < num_random_values:
    parameters = np.round([np.random.rand(), np.random.rand(), (np.random.rand()-0.5)*10, np.random.rand()*10], \
                          decimals = 3)
    # Filter only parameters with complex roots
    a, E, L, Q = parameters
    R = Polynomial([-a**2*Q, 2*L**2+2*Q+2*a**2*E**2-4*a*E*L, a**2*E**2-L**2-Q-a**2, 2, E**2-1])
    radial_roots = R.roots()
    complex_roots = radial_roots[np.iscomplex(radial_roots)]
    
    if parameters[0] != 1 and parameters[1] != 1 and len(complex_roots) > 0:
        orbit_values[i] = parameters
        i += 1

np.savetxt("plunging_orbit_parameters_complex.txt",orbit_values,fmt="%.3f, %.3f, %.3f, %.3f")
times = np.array(range(10))
np.savetxt("plunging_orbit_times.txt",times)

In [7]:
for i, orbit in enumerate(orbit_values):
    session.evaluate(wlexpr("orbit = KerrGeoPlunge[{},{{{},{},{}}}]".format(*orbit)))
    session.evaluate(wlexpr("{Ir, Ir2, Irp, Irm} = Values[orbit[\"RadialIntegrals\"]]"))
    session.evaluate(wlexpr("{tr,phir,tz,phiz} = Values[orbit[\"TrajectoryDeltas\"]]"))
    trajectory_deltas = np.zeros((10,4))
    integrals = np.zeros((10,4))
    for j,t in enumerate(times):
        trajectory_deltas[j] = np.array(session.evaluate(wlexpr(f'N[{{tr[{t}],phir[{t}],tz[{t}],phiz[{t}]}}]')))
        integrals[j] = np.array(session.evaluate(wlexpr(f'N[{{Ir[{t}],Ir2[{t}],Irp[{t}],Irm[{t}]}}]')))
    np.savetxt("plunging_solutions/trajectory"+str(i)+".txt",trajectory_deltas,delimiter=",")
    np.savetxt("plunging_integrals/trajectory"+str(i)+".txt",integrals,delimiter=",")

## Plunging Orbit (Complex Roots)

In [8]:
for i, orbit in enumerate(orbit_values):
    session.evaluate(wlexpr("orbit = KerrGeoPlunge[{},{{{},{},{}}}]".format(*orbit)))
    session.evaluate(wlexpr("{t, r, theta, phi} = orbit[\"Trajectory\"]"))
    trajectory = np.zeros((len(times),4))
    for j,t in enumerate(times):
        trajectory[j] = np.array(session.evaluate(wlexpr(f'N[{{t[{t}],r[{t}],theta[{t}],phi[{t}]}}]')))
    np.savetxt("plunging_orbits_complex/trajectory"+str(i)+".txt",trajectory,delimiter=",")

## Plunging Orbit (Real)

In [11]:
num_random_values = int(100)
orbit_values = np.zeros((num_random_values,4))
i = 0

while i < num_random_values:
    parameters = np.round([np.random.rand(), np.random.rand(), (np.random.rand()-0.5)*10, np.random.rand()*10], \
                          decimals = 3)
    a, E, L, Q = parameters
    R = Polynomial([-a**2*Q, 2*L**2+2*Q+2*a**2*E**2-4*a*E*L, a**2*E**2-L**2-Q-a**2, 2, E**2-1])
    radial_roots = R.roots()
    complex_roots = radial_roots[np.iscomplex(radial_roots)]
    
    if parameters[0] != 1 and parameters[1] != 1 and len(complex_roots) == 0:
        orbit_values[i] = parameters
        i += 1

np.savetxt("plunging_orbit_parameters_real.txt",orbit_values,fmt="%.3f, %.3f, %.3f, %.3f")

In [12]:
for i, orbit in enumerate(orbit_values):
    session.evaluate(wlexpr("orbit = KerrGeoPlunge[{},{{{},{},{}}}]".format(*orbit)))
    session.evaluate(wlexpr("{t, r, theta, phi} = orbit[\"Trajectory\"]"))
    trajectory = np.zeros((len(times),4))
    for j,t in enumerate(times):
        trajectory[j] = np.array(session.evaluate(wlexpr(f'N[{{t[{t}],r[{t}],theta[{t}],phi[{t}]}}]')))
    np.savetxt("plunging_orbits_real/trajectory"+str(i)+".txt",trajectory,delimiter=",")

## Four Velocity

In [14]:
orbit_values = np.genfromtxt("/users/spark59/Documents/KerrGeoPy/tests/data/stable_orbit_values.txt",
                             delimiter=",")
for i, orbit in enumerate(orbit_values):
    session.evaluate(wlexpr("u = KerrGeoFourVelocity[{:},{:},{:},{:}];".format(*orbit)))
    session.evaluate(wlexpr("{ut,ur,utheta,uphi} = Values[u]"))
    trajectory = np.zeros((10,4))
    for j,t in enumerate(times):
        trajectory[j] = np.array(
            session.evaluate(wlexpr(f'N[{{ut[{t}],ur[{t}],utheta[{t}],uphi[{t}]}}]'))
        )
    np.savetxt("four_velocity/trajectory"+str(i)+".txt",trajectory,delimiter=",")